### Импорт библиотек

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

### Чтение файла

In [2]:
df = pd.read_csv('corp_lemm.csv')
df = df[df['text'].notnull()]
df = df.drop(['Unnamed: 0'], axis = 1)

### Анализ тональности

In [3]:
sentiment_mapping = {'bad': 1, 'neutral': 1, 'good': 0}
df['label'] = df['coloring'].map(sentiment_mapping)
df['label'].value_counts()

0    2120
1    1996
Name: label, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,
                        test_size = 0.2,
                        random_state = 12348,
                       )

print(train.shape)
print(test.shape)
for sample in [train, test]:    
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

(3292, 9)
(824, 9)
0.4787363304981774
0.5097087378640777


In [5]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[^А-яЁё]+', ' ', line)
    return line

In [6]:
tfidf = TfidfVectorizer(preprocessor=preprocessing)
tfidf_train = tfidf.fit_transform(train['text'])
tfidf_test = tfidf.transform(test['text'])
kmeans = KMeans(n_clusters=2).fit(tfidf_train)


In [7]:
k_means = kmeans.predict(tfidf_test).tolist()


In [8]:
test['predictions'] = k_means

<ipython-input-8-ba3aa184d0b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predictions'] = k_means


In [9]:
test.head(5)

,text,rated,coloring,date,name,bank,sex,text_lemmatized,label,predictions
2303,"Обратился По вопросу обслуживания карты,сотруд...",4,good,"14 февр, 2022",Юшко Андрей Сергеевич,sovkombank,unkn,обратиться по вопрос обслуживание карта сотруд...,0,0
1184,мая была подана заявка в банк для утверждения ...,2,bad,"21 мая, 2019",Аноним,rossija,unkn,май подать заявка банк утверждение сумма котор...,1,1
1100,ноября в 14. 10 села в ожидании очереди в касс...,2,bad,"26 нояб, 2019",Аноним,primore,unkn,ноябрь село ожидание очередь касса банк прим...,1,1
2317,"Хотела закрыть карту,но уговорили оставить. До...",4,good,"13 февр, 2022",Линара,sovkombank,unkn,хотеть закрыть карта уговорить оставить до се...,0,0
2397,Оформляла продажу квартиры и нужна была банков...,4,good,"28 мая, 2021",Марина Агоповна,ehnergomashbank,unkn,оформлять продажа квартира нужный банковский я...,0,0


In [10]:
pd.options.display.max_colwidth = 10000

In [11]:
cl0 = test.query('predictions == 0')
cl1 = test.query('predictions == 1')


In [12]:
cl0['rated'].value_counts()

5    224
4     94
1     17
3     14
2     11
Name: rated, dtype: int64

In [13]:
cl1['rated'].value_counts()

2    133
3    125
1    120
4     62
5     24
Name: rated, dtype: int64

Понятно, что класс 0 – это положительные отзывы, остальные - отрицательные.

### Оценка работы алгоритма

In [14]:
pred = []
true = []
for i in test['predictions']:
    pred.append(i)
for i in test['label']:
    true.append(i)

In [15]:
tp = 0 
tn = 0 
fp = 0 
fn = 0
t_f = []
for i in range(0, len(pred)):
    if pred[i] == true[i]:
        if pred[i] == 1:
            tp+=1
            t_f.append('tp')
        else:
            tn+=1
            t_f.append('tn')
    elif pred[i] != true[i]:
        if pred[i] == 0:
            fp+=1
            t_f.append('fp')
        else:
            fn+=1
            t_f.append('fn')
        

In [16]:
print(' tp -', tp, '\n', 'tn -', tn, '\n', 'fp -', fp, '\n', 'fn-', fn)

 tp - 378 
 tn - 318 
 fp - 42 
 fn- 86


In [17]:
accuracy = (tp+tn)/len(test)
precision = tp/ (tp+fp)
recall = tp/(tp+fn)
f1 = 2/(1/precision+1/recall)
f_measure = 2*((precision*recall)/(precision+recall))

In [18]:
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)
print('f1-score', f1)
print('f-measure', f_measure)

accuracy 0.8446601941747572
precision 0.9
recall 0.8146551724137931
f1-score 0.8552036199095022
f-measure 0.8552036199095023


### Общие выводы

Значение F-меры на 0,05 ниже, чем у Байесовского классификатора модели машинного обучения с учителем и на 0,08 ниже, чем у алгоритма, основанного на правилах и словарях, при этом обе метрики – полноты и точности оказались ниже.
